In [1]:
pip install linearmodels

In [2]:
import pandas as pd
file="https://github.com/VitorKamada/ECO7110/raw/master/Data/wagepan.dta"
data = pd.read_stata(file)
data[1:10]

,nr,year,agric,black,bus,construc,ent,exper,fin,hisp,poorhlth,hours,manuf,married,min,nrthcen,nrtheast,occ1,occ2,occ3,occ4,occ5,occ6,occ7,occ8,occ9,per,pro,pub,rur,south,educ,tra,trad,union,lwage,d81,d82,d83,d84,d85,d86,d87,expersq
1,13,1981,0,0,0,0,0,2,0,0,0,2320,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,14,0,0,1,1.853060,1,0,0,0,0,0,0,4
2,13,1982,0,0,1,0,0,3,0,0,0,2940,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,14,0,0,0,1.344462,0,1,0,0,0,0,0,9
3,13,1983,0,0,1,0,0,4,0,0,0,2960,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,14,0,0,0,1.433213,0,0,1,0,0,0,0,16
4,13,1984,0,0,0,0,0,5,0,0,0,3071,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,14,0,0,0,1.568125,0,0,0,1,0,0,0,25
5,13,1985,0,0,1,0,0,6,0,0,0,2864,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,14,0,0,0,1.699891,0,0,0,0,1,0,0,36
6,13,1986,0,0,1,0,0,7,0,0,0,2994,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,14,0,0,0,-0.720263,0,0,0,0,0,1,0,49
7,13,1987,0,0,1,0,0,8,0,0,0,2640,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,14,0,0,0,1.669188,0,0,0,0,0,0,1,64
8,17,1980,0,0,0,0,0,4,0,0,0,2484,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,13,0,1,0,1.675962,0,0,0,0,0,0,0,16
9,17,1981,0,0,0,0,0,5,0,0,0,2804,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,13,0,1,0,1.518398,1,0,0,0,0,0,0,25


In [3]:
year = pd.Categorical(data.year)
nr = pd.Categorical(data.nr)
data = data.set_index(['nr', 'year'])
data['year'] = year
data['nr'] = nr
data[1:10]

agric  black  bus  construc  ent  ...  d86  d87  expersq  year  nr
nr year                                    ...                             
13 1981      0      0    0         0    0  ...    0    0        4  1981  13
   1982      0      0    1         0    0  ...    0    0        9  1982  13
   1983      0      0    1         0    0  ...    0    0       16  1983  13
   1984      0      0    0         0    0  ...    0    0       25  1984  13
   1985      0      0    1         0    0  ...    0    0       36  1985  13
   1986      0      0    1         0    0  ...    1    0       49  1986  13
   1987      0      0    1         0    0  ...    0    1       64  1987  13
17 1980      0      0    0         0    0  ...    0    0       16  1980  17
   1981      0      0    0         0    0  ...    0    0       25  1981  17

[9 rows x 44 columns]

In [4]:
from linearmodels.panel import PooledOLS
import statsmodels.api as sm
exog_vars = ['black','hisp','exper','expersq',
           'married', 'educ', 'union', 'year']
exog = sm.add_constant(data[exog_vars])
OLS = PooledOLS(data.lwage, 
 exog).fit(cov_type='clustered', cluster_entity=True)
print(OLS)

                          PooledOLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                        0.1893
Estimator:                  PooledOLS   R-squared (Between):              0.2066
No. Observations:                4360   R-squared (Within):               0.1692
Date:                Wed, Aug 07 2019   R-squared (Overall):              0.1893
Time:                        14:35:16   Log-likelihood                   -2982.0
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      72.459
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                 F(14,4345)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             47.177
                            

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [5]:
from linearmodels.panel import RandomEffects
RE = RandomEffects(data.lwage, 
 exog).fit(cov_type='clustered', cluster_entity=True)
print(RE)

                        RandomEffects Estimation Summary                        
Dep. Variable:                  lwage   R-squared:                        0.1806
Estimator:              RandomEffects   R-squared (Between):              0.1853
No. Observations:                4360   R-squared (Within):               0.1799
Date:                Wed, Aug 07 2019   R-squared (Overall):              0.1828
Time:                        14:35:16   Log-likelihood                   -1622.5
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      68.409
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                 F(14,4345)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             43.681
                            

In [6]:
from linearmodels.panel import PanelOLS
FE = PanelOLS(data.lwage, exog, entity_effects=True, 
 drop_absorbed=True).fit(cov_type='clustered', cluster_entity=True)
print(FE)

                          PanelOLS Estimation Summary                           
Dep. Variable:                  lwage   R-squared:                        0.1806
Estimator:                   PanelOLS   R-squared (Between):             -0.0528
No. Observations:                4360   R-squared (Within):               0.1806
Date:                Wed, Aug 07 2019   R-squared (Overall):              0.0552
Time:                        14:35:16   Log-likelihood                   -1324.8
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      83.851
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                 F(10,3805)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             46.661
                            

/usr/local/lib/python3.6/dist-packages/linearmodels/panel/model.py:1316: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

black, hisp, educ, year.1987

  AbsorbingEffectWarning)


In [7]:
from linearmodels.panel import compare
print(compare({'OLS':OLS,'RE': RE, 'FE': FE}))

                           Model Comparison                          
                                   FE           OLS                RE
---------------------------------------------------------------------
Dep. Variable                   lwage         lwage             lwage
Estimator                    PanelOLS     PooledOLS     RandomEffects
No. Observations                 4360          4360              4360
Cov. Est.                   Clustered     Clustered         Clustered
R-squared                      0.1806        0.1893            0.1806
R-Squared (Within)             0.1806        0.1692            0.1799
R-Squared (Between)           -0.0528        0.2066            0.1853
R-Squared (Overall)            0.0552        0.1893            0.1828
F-statistic                    83.851        72.459            68.409
P-value (F-stat)               0.0000        0.0000            0.0000
=====================     ===========   ===========   ===============
const               

In [0]:
data['experbar'] = data.groupby(nr)['exper'].transform('mean')
data['expersqbar'] = data.groupby(nr)['expersq'].transform('mean')
data['marriedbar'] = data.groupby(nr)['married'].transform('mean')
data['unionbar'] = data.groupby(nr)['union'].transform('mean')

In [9]:
data[1:10]

agric  black  bus  ...  expersqbar  marriedbar  unionbar
nr year                     ...                                  
13 1981      0      0    0  ...        25.5         0.0     0.125
   1982      0      0    1  ...        25.5         0.0     0.125
   1983      0      0    1  ...        25.5         0.0     0.125
   1984      0      0    0  ...        25.5         0.0     0.125
   1985      0      0    1  ...        25.5         0.0     0.125
   1986      0      0    1  ...        25.5         0.0     0.125
   1987      0      0    1  ...        25.5         0.0     0.125
17 1980      0      0    0  ...        61.5         0.0     0.000
   1981      0      0    0  ...        61.5         0.0     0.000

[9 rows x 48 columns]

In [10]:
Exog = ['black','hisp','exper','expersq','married', 'educ',
        'union', 'experbar', 'expersqbar', 'marriedbar',
        'unionbar', 'd81','d82','d83','d84','d85','d86']
Exog = sm.add_constant(data[Exog])

CRE = RandomEffects(data.lwage,
    Exog).fit(cov_type='clustered', cluster_entity=True)
print(CRE)

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


                        RandomEffects Estimation Summary                        
Dep. Variable:                  lwage   R-squared:                        0.1855
Estimator:              RandomEffects   R-squared (Between):              0.2192
No. Observations:                4360   R-squared (Within):               0.1806
Date:                Wed, Aug 07 2019   R-squared (Overall):              0.2013
Time:                        14:35:17   Log-likelihood                   -1608.1
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      58.165
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                 F(17,4342)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             40.705
                            